# Import packages and get authenticated

In [1]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [0]:
import numpy as np

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc( 'savefig', facecolor = 'white' )

import pandas as pd
pd.set_option('display.max_columns', 500)
from tqdm import tqdm_notebook as tqdm
import os
import sys
from IPython.display import display

import time
import datetime
from datetime import datetime
import json

from sklearn.decomposition import PCA

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

In [0]:
def data_saver(data, name, outputdir):
    """ usage: data_saver(df_merged_interp_alldicts, 'data', outputdir)"""
    outputdir_data = os.path.join(outputdir, name+'.npz')
    print('outputdir for {}:'.format(name), outputdir_data)
    np.savez(outputdir_data, data=data, allow_pickle=True)
    loaded_data = np.load(outputdir_data, allow_pickle=True)['data']
#     loaded_data = np.load(outputdir_data, allow_pickle=True)['data']
    print('Are {} save and loadded correctly? '.format(name), np.array_equal(loaded_data, data))
    print('')
    
def data_loader(name, inputdir):
    """ usage: data = data_loader('data', outputdir)"""
    inputdir_data = os.path.join(inputdir, name+'.npz')
    data = np.load(inputdir_data, allow_pickle=True)['data']
    return data

In [0]:
class FallDataset(Dataset):
  def __init__(self, data, labels):
      self.data = torch.FloatTensor(data)
      self.labels = torch.FloatTensor(labels)
      # self.data = torch.LongTensor(data)
      # self.labels = torch.LongTensor(labels)

  def __getitem__(self, index):
      x = self.data[index,:,:]
      y = self.labels[index]
      return x, y

  def __len__(self):
      return len(self.data)

In [0]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10, input_dim=50):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv1d(3, 16, kernel_size=3, stride=1, padding=2),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv1d(16, 32, kernel_size=3, stride=1, padding=2),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv1d(32, 64, kernel_size=3, stride=1, padding=2),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2))
        
        cnn_layer1_dim = (input_dim+2*2-1*(3-1)-1)+1
        pool_layer1_dim = (cnn_layer1_dim-1*(2-1)-1)/2+1

        cnn_layer2_dim = (pool_layer1_dim+2*2-1*(3-1)-1)+1
        pool_layer2_dim = (cnn_layer2_dim-1*(2-1)-1)/2+1

        cnn_layer3_dim = (pool_layer2_dim+2*2-1*(3-1)-1)+1
        pool_layer3_dim = (cnn_layer3_dim-1*(2-1)-1)/2+1

        # print('cnn_layer1_dim:', cnn_layer1_dim)
        # print('pool_layer1_dim:', pool_layer1_dim)
        # print('cnn_layer2_dim:', cnn_layer2_dim)
        # print('pool_layer2_dim:', pool_layer2_dim)
        # print('cnn_layer3_dim:', cnn_layer3_dim)
        # print('pool_layer3_dim:', pool_layer3_dim)
        # fc_dim = int(((((input_dim)+2*2-1)/2+2*2-1)/2+2*2-1)/2*64)
        self.fc = nn.Linear(int(pool_layer3_dim)*64, num_classes)
        
    def forward(self, x):
      out1 = self.layer1(x.float())
      # print('out1 size:', out1.size())
      out2 = self.layer2(out1)
      # print('out2 size:', out2.size())
      out3 = self.layer3(out2)
      # print('out3 size:', out3.size())
      out3 = out3.reshape(out3.size(0), -1)
      # print('out3 size:', out3.size())
      out4 = self.fc(out3)
      # print('x, out1, out2, out 3, out4 size',  x.size(), out1.size(), out2.size(), out3.size(), out4.size())
      return out4

In [0]:
# aaa = ConvNet(num_classes=10, input_dim=50)

In [0]:
# int((input_dim+2*2-1)/2*64)

# Exporting function for model performance, notebook parameters, and model

## set export filepath

In [0]:
# outputdir = '/content/drive/My Drive/中研院/data_mic/stage2_modeloutput/{}/{}/'.format(dataset_name, sensor_loc)

# if not os.path.exists(outputdir):
#     os.makedirs(outputdir)
# print('outputdir for stage2 output:', outputdir)

## export model performance

In [0]:
def export_perofmance(df_performance, CV_n, outputdir):
  df_performance.loc['mean'] = df_performance.iloc[0:CV_n-1].mean()
  df_performance.loc['std'] = df_performance.iloc[0:CV_n-1].std()
  print('show df_performance')
  display(df_performance)

  print('validation loss: {:.4f}±{:.4f}'.format(df_performance.loc['mean']['val_loss'], df_performance.loc['std']['val_loss']))
  print('validation acc: {:.4f}±{:.4f}'.format(df_performance.loc['mean']['val_acc'], df_performance.loc['std']['val_acc']))

  df_performance.to_csv(outputdir+'df_performance.csv')

## export model using torch.save and validate saved model

In [0]:
def export_model(model, classes_n, input_dim, device, outputdir):
  # Save
  torch.save(model.state_dict(), outputdir+'model')
  # Load
  loaded_model = ConvNet(num_classes=classes_n, input_dim=input_dim).to(device).float()
  loaded_model.load_state_dict(torch.load(outputdir+'model'))

  save_error = 0
  for p1, p2 in zip(model.parameters(), loaded_model.parameters()):
    if p1.data.ne(p2.data).sum() > 0:
      print('model not successfully saved')
      save_error = 1
      break

  if save_error == 0:
    print('model saved successfully')

# Preparing dataset and fit it to a convnet

In [0]:
# dataset_name = 'UMAFall'
# sensor_loc = 'waist'
# sensor_loc = 'wrist'
# sensor_loc = 'leg'
# sensor_loc = 'chest'
# sensor_loc = 'ankle'
dataset_name = 'UPFall'
# sensor_loc = 'wrist'
# sensor_loc = 'rightpocket'
# sensor_loc = 'neck'
sensor_loc = 'belt'
# sensor_loc = 'ankle'

inputdir = '/content/drive/My Drive/中研院/data_mic/stage1_preprocessed/{}/{}/'.format(dataset_name, sensor_loc)

classes_n = 2
CV_n = 5

# Start CV training and validation in a big phat loop

In [0]:
def model_fitting(CV_n, classes_n, sensor_loc, dataset_name, inputdir): 
  # it's big phat loop i don't like it qq
  df_performance = pd.DataFrame(columns=['i_CV','train_loss','train_acc','val_loss','val_acc'])

  for i_CV in range(CV_n):
    # 1. prepare dataset
    train_inputdir = inputdir+'/CV{}/train'.format(i_CV)
    val_inputdir = inputdir+'/CV{}/val'.format(i_CV)

    train_data = data_loader('data', train_inputdir).transpose(2,1,0)
    val_data = data_loader('data', val_inputdir).transpose(2,1,0)

    train_labels = data_loader('labels', train_inputdir)
    val_labels = data_loader('labels', val_inputdir)

    train_i_sub = data_loader('i_sub', train_inputdir)
    val_i_sub = data_loader('i_sub', val_inputdir)

    print('train_data shape:', train_data.shape)
    print('val_data shape:', val_data.shape)

    train_size = train_labels.shape[0]
    val_size = val_labels.shape[0]
    input_dim = train_data.shape[2]

    # convert labels from multi-class activities to binary (fall/ADL)
    train_labels_binary = ((train_labels==10)|(train_labels==11)|(train_labels==12)).astype(int)
    val_labels_binary = ((val_labels==10)|(val_labels==11)|(val_labels==12)).astype(int)

    train_dataset = FallDataset(train_data, train_labels_binary)
    val_dataset = FallDataset(val_data, val_labels_binary)
    # data loader
    batch_size = 4
    learning_rate = 0.001

    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                              batch_size=batch_size, 
                                              shuffle=True)

    val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                              batch_size=batch_size, 
                                              shuffle=False)

    # 2. prepare model
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    model = ConvNet(num_classes=classes_n, input_dim=input_dim).to(device).float()

    # loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # test model on a batch
    try:
      out = model(train_dataset.data[0:batch_size,:,:])
      model_outdim = out.data.numpy().shape
    except:
      print('Warning: model cannot read input')

    print('{} model architecture: '.format(model.__class__.__name__))
    print(model)


    # 3. fit the model
    num_epochs = 10
    total_step = len(train_loader)
    for epoch in range(num_epochs):
      total_train_loss = 0
      train_TPTF = 0
      for i, (data, labels) in enumerate(train_loader):
        data = data.to(device)
        labels = labels.to(device).long()

        # Forward pass
        outputs = model(data)
        train_loss = criterion(outputs, labels)
        total_train_loss += train_loss.data.numpy()
        
        out_sigmoid = torch.sigmoid(outputs).data.numpy()
        train_pred = np.argmax(out_sigmoid, 1)
        train_TPTF += (train_pred==labels.data.numpy()).sum()
        # train_pred = print(np.argmax(F.sigmoid(outputs))


        # Backward and optimize
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        # if (i+1) % 5 == 0:
        #     print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.8f}' 
        #             .format(epoch+1, num_epochs, i+1, total_step, train_loss.data.numpy()/labels.size()[0]))

      total_val_loss = 0
      val_TPTF = 0
      for i, (data, labels) in enumerate(val_loader):
        data = data.to(device)
        labels = labels.to(device).long()
        
        #Forward pass
        val_outputs = model(data)
        val_loss = criterion(val_outputs, labels)
        total_val_loss += val_loss.data.numpy()

        out_sigmoid = torch.sigmoid(val_outputs).data.numpy()
        val_pred = np.argmax(out_sigmoid, 1)
        val_TPTF += (val_pred==labels.data.numpy()).sum()
        # print(val_TPTF, len(val_loader))
          
      train_loss = total_train_loss/train_size
      train_acc = train_TPTF/train_size
      val_loss = total_val_loss/val_size
      val_acc = val_TPTF/val_size


      print('Epoch {}'.format(epoch+1))
      print('Train Loss: {:.6f}, Train ACC: {:.6f}, Val loss = {:.6f}, Val ACC: {:.6f}'.
            format(train_loss, train_acc, val_loss, val_acc))
    
    # 4. store the performance of the model at the last epoch
    df_performance.loc[i_CV] = [i_CV, train_loss, train_acc, val_loss, val_acc]

  outputdir = '/content/drive/My Drive/中研院/data_mic/stage2_modeloutput/{}/{}/'.format(dataset_name, sensor_loc)
  if not os.path.exists(outputdir):
      os.makedirs(outputdir)
  print('outputdir for stage2 output:', outputdir)

  # 5. export model performance as df
  export_perofmance(df_performance, CV_n, outputdir)

  # 6. export notebook parameters as dict
  # datetime object containing current date and time
  now = datetime.now()
  dt_string = now.strftime("%Y/%m/%d %H:%M:%S")
  samples_n = train_size + val_size

  param_dict = {
      'CV_n': CV_n,
      'samples_n': samples_n,
      'classes_n': classes_n,
      'model_name': model.__class__.__name__,
      'dataset_name': dataset_name,
      'sensor_loc': sensor_loc,
      'date': dt_string,
      'batch_size': batch_size,
      'input_dim': (batch_size, train_dataset.data.size()[1], train_dataset.data.size()[2]),
      'output_dim': train_dataset.labels[0:batch_size].data.numpy().shape,
      'label_dim': CV_n,
  }
  print(param_dict)

  with open(outputdir+'notebook_param.json', 'w') as fp:
    json.dump(param_dict, fp)

  export_model(model, classes_n, input_dim, device, outputdir)

In [0]:
datasets_sensor_dict = {
    'UMAFall': ['waist', 'wrist', 'leg', 'chest', 'ankle'],
    'UPFall': ['wrist', 'rightpocket', 'neck', 'belt', 'ankle']
}

In [19]:
inputdir = '/content/drive/My Drive/中研院/data_mic/stage1_preprocessed/{}/{}/'.format(dataset_name, sensor_loc)
classes_n = 2
CV_n = 5

for key in datasets_sensor_dict.keys():
  dataset_name = key
  for sensor_loc in datasets_sensor_dict[dataset_name]:
    model_fitting(CV_n, classes_n, sensor_loc, dataset_name, inputdir)

train_data shape: (384, 3, 71)
val_data shape: (117, 3, 71)
ConvNet model architecture: 
ConvNet(
  (layer1): Sequential(
    (0): Conv1d(3, 16, kernel_size=(3,), stride=(1,), padding=(2,))
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv1d(16, 32, kernel_size=(3,), stride=(1,), padding=(2,))
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(2,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=640, out_features=

,i_CV,train_loss,train_acc,val_loss,val_acc
0,0.000000,0.000076,1.0,0.002379,1.000000
1,1.000000,0.000279,1.0,0.000508,1.000000
2,2.000000,0.000065,1.0,0.000462,1.000000
3,3.000000,0.000139,1.0,0.000344,1.000000
4,4.000000,0.000061,1.0,0.007442,0.988506
mean,1.500000,0.000140,1.0,0.000923,1.000000
std,1.290994,0.000098,0.0,0.000973,0.000000


validation loss: 0.0009±0.0010
validation acc: 1.0000±0.0000
{'CV_n': 5, 'samples_n': 501, 'classes_n': 2, 'model_name': 'ConvNet', 'dataset_name': 'UMAFall', 'sensor_loc': 'waist', 'date': '2020/02/10 10:56:26', 'batch_size': 4, 'input_dim': (4, 3, 71), 'output_dim': (4,), 'label_dim': 5}
model saved successfully
train_data shape: (384, 3, 71)
val_data shape: (117, 3, 71)
ConvNet model architecture: 
ConvNet(
  (layer1): Sequential(
    (0): Conv1d(3, 16, kernel_size=(3,), stride=(1,), padding=(2,))
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv1d(16, 32, kernel_size=(3,), stride=(1,), padding=(2,))
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Seq

,i_CV,train_loss,train_acc,val_loss,val_acc
0,0.000000,0.000064,1.0,0.003633,0.991453
1,1.000000,0.000784,1.0,0.000605,1.000000
2,2.000000,0.000147,1.0,0.000236,1.000000
3,3.000000,0.000040,1.0,0.000053,1.000000
4,4.000000,0.000059,1.0,0.010966,0.977011
mean,1.500000,0.000259,1.0,0.001132,0.997863
std,1.290994,0.000353,0.0,0.001683,0.004274


validation loss: 0.0011±0.0017
validation acc: 0.9979±0.0043
{'CV_n': 5, 'samples_n': 501, 'classes_n': 2, 'model_name': 'ConvNet', 'dataset_name': 'UMAFall', 'sensor_loc': 'wrist', 'date': '2020/02/10 10:57:03', 'batch_size': 4, 'input_dim': (4, 3, 71), 'output_dim': (4,), 'label_dim': 5}
model saved successfully
train_data shape: (384, 3, 71)
val_data shape: (117, 3, 71)
ConvNet model architecture: 
ConvNet(
  (layer1): Sequential(
    (0): Conv1d(3, 16, kernel_size=(3,), stride=(1,), padding=(2,))
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv1d(16, 32, kernel_size=(3,), stride=(1,), padding=(2,))
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Seq

...

validation loss: 0.0013±0.0018
validation acc: 0.9979±0.0043
{'CV_n': 5, 'samples_n': 501, 'classes_n': 2, 'model_name': 'ConvNet', 'dataset_name': 'UPFall', 'sensor_loc': 'neck', 'date': '2020/02/10 11:00:47', 'batch_size': 4, 'input_dim': (4, 3, 71), 'output_dim': (4,), 'label_dim': 5}
model saved successfully
train_data shape: (384, 3, 71)
val_data shape: (117, 3, 71)
ConvNet model architecture: 
ConvNet(
  (layer1): Sequential(
    (0): Conv1d(3, 16, kernel_size=(3,), stride=(1,), padding=(2,))
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv1d(16, 32, kernel_size=(3,), stride=(1,), padding=(2,))
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Seque

,i_CV,train_loss,train_acc,val_loss,val_acc
0,0.000000,0.000053,1.0,0.002438,1.0
1,1.000000,0.000528,1.0,0.000409,1.0
2,2.000000,0.000470,1.0,0.000012,1.0
3,3.000000,0.000255,1.0,0.000038,1.0
4,4.000000,0.000047,1.0,0.003080,1.0
mean,1.500000,0.000327,1.0,0.000724,1.0
std,1.290994,0.000217,0.0,0.001157,0.0


validation loss: 0.0007±0.0012
validation acc: 1.0000±0.0000
{'CV_n': 5, 'samples_n': 501, 'classes_n': 2, 'model_name': 'ConvNet', 'dataset_name': 'UPFall', 'sensor_loc': 'belt', 'date': '2020/02/10 11:01:29', 'batch_size': 4, 'input_dim': (4, 3, 71), 'output_dim': (4,), 'label_dim': 5}
model saved successfully
train_data shape: (384, 3, 71)
val_data shape: (117, 3, 71)
ConvNet model architecture: 
ConvNet(
  (layer1): Sequential(
    (0): Conv1d(3, 16, kernel_size=(3,), stride=(1,), padding=(2,))
    (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv1d(16, 32, kernel_size=(3,), stride=(1,), padding=(2,))
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Seque

,i_CV,train_loss,train_acc,val_loss,val_acc
0,0.000000,0.007019,0.992188,0.055670,0.957265
1,1.000000,0.000159,1.000000,0.000847,1.000000
2,2.000000,0.000127,1.000000,0.000019,1.000000
3,3.000000,0.000182,1.000000,0.000082,1.000000
4,4.000000,0.000033,1.000000,0.004337,0.988506
mean,1.500000,0.001872,0.998047,0.014154,0.989316
std,1.290994,0.003431,0.003906,0.027679,0.021368


validation loss: 0.0142±0.0277
validation acc: 0.9893±0.0214
{'CV_n': 5, 'samples_n': 501, 'classes_n': 2, 'model_name': 'ConvNet', 'dataset_name': 'UPFall', 'sensor_loc': 'ankle', 'date': '2020/02/10 11:02:06', 'batch_size': 4, 'input_dim': (4, 3, 71), 'output_dim': (4,), 'label_dim': 5}
model saved successfully
